# kaggle study 30일차(New-York taxi)
코드출처 :https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-zillow-prize

이 노트에서, 우리는 zillow 경품 경진대회에서 주어진 데이터를 살펴보도록 하겠습니다. 데이터를 자세히 살펴보기 전에 경쟁업체에 대해 좀 더 자세히 알아보십시오.

zillow:

Zillow는 2006년에 설립된 온라인 부동산 데이터베이스 회사입니다 - 위키백과

추정치:

"예상치"는 각 속성에 대한 수백 개의 데이터 포인트를 분석하는 750만 개의 통계 및 기계 학습 모델을 기반으로 추정된 집 값입니다. 또한 중앙 오차 한계를 지속적으로 개선함으로써(시작 시 14%에서 현재 5%로),

목표:

Z 추정 잔차 오류를 개선하기 위한 모형 작성입니다.

그 대회는 두 단계로 나뉘어 있습니다. 이 공개 경쟁은 2018년 1월까지 계속되며 상금은 5만 달러입니다. 이번 대회에서는 상이하기 때문에 상 세부 정보와 경기 개요에 대해 반드시 읽어보시기 바랍니다.

먼저 필요한 모듈을 가져오겠습니다.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [ ]:
train_df = pd.read_csv("C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/train_2016_v2.csv", engine='python', parse_dates=["transactiondate"])
train_df.shape

In [ ]:
train_df.head()

__Logerror__:  
이 경기의 대상 변수는 "Logerror" 필드입니다. 먼저 이 분야에 대한 분석을 해보겠습니다.

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(range(train_df.shape[0]), np.sort(train_df.logerror.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('logerror', fontsize=12)
plt.show()

양쪽 끝에 특이치가 있는 것이 보기 좋습니다!

특이치를 제거한 다음 히스토그램 그림을 동일한 방법으로 수행하겠습니다.

In [ ]:
ulimit = np.percentile(train_df.logerror.values, 99)
llimit = np.percentile(train_df.logerror.values, 1)
train_df['logerror'][train_df['logerror']>ulimit] = ulimit
train_df['logerror'][train_df['logerror']<llimit] = llimit

plt.figure(figsize=(12,8))
sns.distplot(train_df.logerror.values, bins=50, kde=False)
plt.xlabel('logerror', fontsize=12)
plt.show()

로그 오류에 대한 정규 분포가 좋군요

__Transaction Date__:

이제 날짜 필드를 살펴보겠습니다. 우선 매월 거래 건수를 확인해 보겠습니다.

In [ ]:
train_df['transaction_month'] = train_df['transactiondate'].dt.month

cnt_srs = train_df['transaction_month'].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[3])
plt.xticks(rotation='vertical')
plt.xlabel('Month of transaction', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

데이터 페이지에서도 볼 수 있듯이 열차 데이터는 2016년 10월 15일 이전의 모든 트랜잭션과 2016년 10월 15일 이후의 일부 트랜잭션을 포함합니다.

그래서 지난 3개월 동안 짧은 막대가 있습니다.  
  
__Parcel Id__:

In [ ]:
(train_df['parcelid'].value_counts().reset_index())['parcelid'].value_counts()

따라서 대부분의  parcel ids는 데이터 세트에 한 번만 나타납니다.

__Properties 2016__:

이제 properties_2016 파일을 살펴보겠습니다.

In [ ]:
prop_df = pd.read_csv("C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/zillow/properties_2016.csv",engine='python')
prop_df.shape

In [ ]:
prop_df.head()

In [ ]:
missing_df = prop_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

우선 위도 및 경도 변수를 살펴보겠습니다.

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=prop_df.latitude.values, y=prop_df.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

데이터 페이지에서 2016년 3개 카운티(Los Angeles, Orange 및 Ventura, California)의 부동산 전체 목록을 제공합니다.

열차에는 약 90275개의 행이 있지만 속성 파일에는 약 2,985,217개의 행이 있습니다. 두 개의 파일을 병합한 다음 분석을 수행하겠습니다.

In [ ]:
train_df = pd.merge(train_df, prop_df, on='parcelid', how='left')
train_df.head()

이제 다양한 유형의 변수의 dtype을 살펴보겠습니다.

In [ ]:
pd.options.display.max_rows = 65

dtype_df = train_df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df

거의 모든 변수가 개체(범주형) 변수가 거의 없는 부동 변수입니다. 계산해 보겠습니다.

In [ ]:
dtype_df.groupby("Column Type").aggregate('count').reset_index()

이제 이 새로운 병합된 데이터 세트의 Null 수를 확인해 보겠습니다.

In [ ]:
missing_df = train_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count']  / train_df.shape[0]
missing_df[missing_df['missing_ratio'] > 0.999]

네 개의 열에 99.9%의 결측값이 있습니다.!

__Univariate Analysis(일변량 분석)__:

변수가 너무 많으므로 먼저 'float' 변수만 취한 다음 대상 변수와의 상관 관계를 파악하여 변수들이 어떻게 연관되어 있는지 알아보겠습니다.

In [ ]:
mean_values = train_df.mean(axis=0)
train_df.fillna(mean_values, inplace=True)
train_df_new = train_df
x_cols = [col for col in train_df_new.columns 
          if col not in ['logerror'] if train_df_new[col].dtype=='float64']

labels = []
values = []

for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(train_df_new[col].values, 
                             train_df.logerror.values)[0,1])
corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')

ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12,40))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='y')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation= 'horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficeint of the variables")
plt.show()

지정된 변수 집합과 대상 변수의 상관 관계는 전반적으로 낮습니다.

이 그래프의 맨 위에 상관 값이 없는 변수는 거의 없습니다. 내 생각에 그들은 하나의 고유한 값만 가지고 있기 때문에 상관 관계가 없는 것 같아요. 같은 내용을 확인해 보겠습니다.

In [ ]:
corr_zero_cols = ['assessmentyear', 'storytypeid', 'pooltypeid2', 'pooltypeid7', 'pooltypeid10', 'poolcnt', 'decktypeid', 'buildingclasstypeid']
for col in corr_zero_cols:
    print(col, len(train_df_new[col].unique()))

상관 관계가 높은 값을 가진 변수를 취하여 분석을 해보겠습니다.

In [ ]:
corr_df_sel = corr_df[(corr_df['corr_values']>0.02) | (corr_df['corr_values'] < -0.01)]
corr_df_sel

In [ ]:
cols_to_use = corr_df_sel.col_labels.tolist()

temp_df = train_df[cols_to_use]
corrmat = temp_df.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))

sns.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

중요한 변수들 자체는 매우 높은 상관관계를 가지고 있습니다. 이제 각 항목을 살펴보겠습니다.

__Finished SquareFeet 12:__:

완성된 정사각형 피트 12가 로그 오류에 따라 어떻게 변하는지 살펴보겠습니다.

In [ ]:
col = "finishedsquarefeet12"
ulimit = np.percentile(train_df[col].values, 99.5)
llimit = np.percentile(train_df[col].values, 0.5)
train_df[col][train_df[col]>ulimit] = ulimit
train_df[col][train_df[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=train_df.finishedsquarefeet12.values, y=train_df.logerror.values, size=10, color=color[4])
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Finished Square Feet 12', fontsize=12)
plt.title("Finished square feet 12 Vs Log error", fontsize=15)
plt.show()

완성된 정사각형 피트 12 변수가 증가함에 따라 더 큰 오차 범위가 좁아지는 것 같습니다. 아마 더 큰 집은 예측하기 쉬울까요?  
__Calculated finished square feet__:

In [ ]:
col = "calculatedfinishedsquarefeet"
ulimit = np.percentile(train_df[col].values, 99.5)
llimit = np.percentile(train_df[col].values, 0.5)
train_df[col][train_df[col]>ulimit] = ulimit
train_df[col][train_df[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=train_df.calculatedfinishedsquarefeet.values, y=train_df.logerror.values, size=10, color=color[5])
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Calculated finished square feet', fontsize=12)
plt.title("Calculated finished square feet Vs Log error", fontsize=15)
plt.show()

여기도 분포가 이전 분포와 매우 유사합니다. 두 변수 사이의 상관 관계도 높은 것은 당연합니다.

__Bathroom Count__:

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="bathroomcnt", data=train_df)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Bathroom', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Bathroom count", fontsize=15)
plt.show()

욕실 수에는 흥미로운 2.279 값이 있습니다.

편집: MihwaHan이 코멘트에서 지적했듯이, 이것은 평균값입니다:)

이제 이를 기반으로 로그 오류가 어떻게 변경되는지 확인해 보겠습니다.

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="bathroomcnt", y='logerror',data=train_df)
plt.ylabel('Log error', fontsize=12)
plt.xlabel('Bathroom Count', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("How long error changes with bathroom count?", fontsize=15)
plt.show()

__Bedroom count__:

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="bedroomcnt",data=train_df)
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('Bedroom Count', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Bedroom count", fontsize=15)
plt.show()

3.03은 Null 값을 대체한 평균 값입니다.

In [ ]:
train_df['bedroomcnt'][train_df['bedroomcnt']>7] = 7
plt.figure(figsize=(12,8))
sns.violinplot(x='bedroomcnt', y='logerror', data=train_df)
plt.xlabel('Bedroom count', fontsize=12)
plt.ylabel('Log Error', fontsize=12)
plt.show()

In [ ]:
col = "taxamount"
ulimit = np.percentile(train_df[col].values, 99.5)
llimit = np.percentile(train_df[col].values, 0.5)
train_df[col][train_df[col]>ulimit] = ulimit
train_df[col][train_df[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=train_df['taxamount'].values, y=train_df['logerror'].values, size=10, color='g')
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Tax Amount', fontsize=12)
plt.title("Tax Amount Vs Log error", fontsize=15)
plt.show()

__YearBuilt__:  
연도별 변수에 따라 오차가 어떻게 변하는지 살펴보겠습니다.

In [ ]:
from plotnine  import *
ggplot(aes(x=train_df['yearbuilt'], y=train_df['logerror']), data=train_df) + \
    geom_point(color='steelblue', size=1) + \
    stat_smooth()

건설 연도와 관련하여 소폭의 증가 추세가 나타나고 있습니다.

이제 위도 및 경도에 따라 로거 오차가 어떻게 변하는지 살펴보겠습니다.

In [ ]:
ggplot(aes(x='latitude', y='longitude', color='logerror'), data=train_df) + \
    geom_point() + \
    scale_color_gradient(low = 'red', high = 'blue')

적어도 육안으로는 위도나 경도와 관련하여 보이는 주머니가 없습니다.

양의 상관 관계가 가장 높고 음의 상관 관계가 가장 높은 변수를 사용하여 몇 가지 가시적인 패턴을 볼 수 있는지 알아보겠습니다.

In [ ]:
ggplot(aes(x='finishedsquarefeet12', y='taxamount', color='logerror'), data=train_df) + \
    geom_point(alpha=0.7) + \
    scale_color_gradient(low = 'pink', high = 'blue')

우리는 일변량 분석을 통해 중요한 변수를 이해했습니다. 하지만 이것은 독립적인 기준이며 또한 선형성 가정도 가지고 있습니다. 이제 추가 트리 모델을 구축하여 중요한 변수를 얻을 수 있는 비선형 모델을 구축하겠습니다.

In [ ]:
train_y = train_df['logerror'].values
cat_cols = ["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", "fireplaceflag", "taxdelinquencyflag"]
train_df = train_df.drop(['parcelid', 'logerror', 'transactiondate', 'transaction_month']+cat_cols, axis=1)
feat_names = train_df.columns.values

from sklearn import ensemble
model = ensemble.ExtraTreesRegressor(n_estimators=25, max_depth=30, max_features=0.3, n_jobs=-1, random_state=0)
model.fit(train_df, train_y)

## plot the importances ##
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importances)[::-1][:20]

plt.figure(figsize=(12,12))
plt.title("Feature importances")
plt.bar(range(len(indices)), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical')
plt.xlim([-1, len(indices)])
plt.show()

In [ ]:
import xgboost as xgb
xgb_params = {'eta': 0.05,
              'max_depth': 8,
              'subsample': 0.7,
              'colsample_bytree': 0.7,
              'objective': 'reg:linear',
              'silent': 1,
              'seed' : 0}
dtrain = xgb.DMatrix(train_df, train_y, feature_names=train_df.columns.values)
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=50)

# plot the important features #
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(model, max_num_features=50, height=0.8, ax=ax)
plt.show()